In [84]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [2]:
import textnet

In [129]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [4]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [85]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ECCE_AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KPpPaeX215HR_fVrakvJp8aB6oZDhHFTcBw0MKLw6as/edit?usp=sharing")

In [94]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201127.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",1982,"[[ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή, μελί..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [95]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14601227

In [96]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [97]:
subcorpora_dict = {
    "Pagan (8th c. BCE - 4th c. CE)" : AGT[(AGT["provenience"]=="pagan") & (AGT["date_avr"] < 4)],
    "Pagan (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")]}

In [98]:
subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "doccount" : doccount, "n_sentences": sents_len,"sentences" : sents})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,doccount,n_sentences,sentences
0,Pagan (8th c. BCE - 4th c. CE),12965219,774,1039409,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,Pagan (1st - 4th c. CE),8692448,411,723956,"[[γεωγραφία, σόσσιε, σενεκίων, ἱστορικός, διαφ..."
2,Septuagint,799083,55,57856,"[[εἰμί, ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ..."
3,Philo of Alexandria,643011,31,82966,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,Titus Flavius Josephus,469580,4,19707,"[[ἱστορία, συγγράφω, βούλομαι, αὐτός, ὁράω, σπ..."
5,Paul of Tarsus,24066,7,1632,"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."
6,New Testament,137787,27,8698,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
7,Christian (1st - 4th c. CE),2975644,131,291084,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."


# term2term & term2doc matraces with sklearn - Paul as example

In [99]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [100]:
len(docs)

1632

In [101]:
AGT_C = subcorpora_dict["Christian (1st - 4th c. CE)"]
len(AGT_C)

131

In [102]:
AGT_C.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
385,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,18288,tlg0031a,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,[ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",8658
386,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,11274,tlg0031b,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ . Καθὼς γέγ...,"[ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ ., Καθὼς γέ...",790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...","[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσα...",5298
387,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,[ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,1274,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πληρ...",9195
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...","[ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...",1164,"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θ...",7060
389,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...","[τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...",960,"[[πρότερος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἔρδω,...","[πρότερος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἔρδω, ...",9096


In [103]:
author = AGT_C["author"][10]
author

'Paul of Tarsus'

In [104]:
sub_df = AGT_C[AGT_C["author_id"]=="tlg0031john"]
sub_df["date_probs"]

388    {'0.5': 1}
407    {'0.5': 1}
408    {'0.5': 1}
409    {'0.5': 1}
Name: date_probs, dtype: object

In [105]:
get_flat_sentences(sub_df["lemmatized_sentences"])

[['εἰμί',
  'ἀρχή',
  'εἰμί',
  'λόγος',
  'λόγος',
  'εἰμί',
  'θεός',
  'θεός',
  'εἰμί',
  'λόγος'],
 ['οὗτος', 'ἀρχή', 'θεός'],
 ['πᾶς', 'γίγνομαι', 'γίγνομαι', 'εἷς'],
 ['γίγνομαι', 'ζωή', 'εἰμί', 'ζωή', 'φώς', 'ἄνθρωπος'],
 ['φώς', 'σκοτία', 'φαίνω', 'σκοτία', 'καταλαμβάνω'],
 ['γίγνομαι', 'ἄνθρωπος', 'ἀποστέλλω', 'θεός', 'ὄνομα', 'ἰωάνης'],
 ['οὗτος',
  'ἔρχομαι',
  'μαρτυρία',
  'μαρτύρομαι',
  'φώς',
  'πᾶς',
  'πιστεύω',
  'αὐτοῦ.'],
 ['εἰμί', 'ἐκεῖνος', 'φώς', 'μαρτύρομαι', 'φώς'],
 ['φώς', 'ἀληθινός', 'φωτίζω', 'ἄνθρωπος', 'ἔρχομαι', 'κόσμος'],
 ['κόσμος', 'εἰμί', 'κόσμος', 'γίγνομαι', 'κόσμος', 'γιγνώσκω'],
 ['ἴδιος', 'ἔρχομαι', 'ἴδιος', 'παραλαμβάνω'],
 ['λαμβάνω',
  'δίδωμι',
  'ἐξουσία',
  'τέκνον',
  'θεός',
  'γίγνομαι',
  'πιστεύω',
  'ὄνομα',
  'αἷμα',
  'θέλημα',
  'σάρξ',
  'θέλημα',
  'ἀνήρ',
  'θεός',
  'γεννάω'],
 ['λόγος',
  'σάρξ',
  'γίγνομαι',
  'σκηνόω',
  'θεάομαι',
  'δόξα',
  'δόξα',
  'μονογενής',
  'πατήρ',
  'πλήρης',
  'χάρις',
  'ἀλήθεια'],
 [],
 [

In [106]:
probs_dict = {}
for d in sub_df["date_probs"]:
    probs_dict.update(d)
probs_dict

{'0.5': 1}

# Comparing All Christian Authors

In [183]:
def grouping_gnt(author_id):
    if author_id == "tlg0031a":
        author_id = "tlg0031matt"
    if author_id == "tlg0031b":
        author_id = "tlg0031mark"
    if re.match("tlg0031\w?$", author_id):
        return "tlg0031rest"
    else:
        return author_id
new_author_ids = AGT_C["author_id"].apply(grouping_gnt)

AGT_C["author_id"] = new_author_ids

In [184]:
authors_dicts = []
for author_id in AGT_C["author_id"].unique():
    sub_df = AGT_C[AGT_C["author_id"]==author_id]
    authors_dict = {}
    authors_dict["author_id"] = author_id
    authors_dict["author"] = sub_df["author"][0]
    authors_dict["n_titles"] = len(sub_df)
    authors_dict["titles"] = sub_df["title"].tolist()
    authors_dict["filenames"] = sub_df["filename"].tolist()
    authors_dict["wordcount"] = sub_df["wordcount"].sum()
    authors_dict["lemmata_wordcount"] = sub_df["lemmata_wordcount"].sum()
    authors_dict["n_sentences"] = sub_df["n_sentences"].sum()
    probs_dict = {}
    for d in sub_df["date_probs"]:
        probs_dict.update(d)
    authors_dict["date_probs"] = probs_dict
    authors_dict["lemmata_lists"] = sub_df["lemmata"].tolist()
    authors_dict["lemmatized_sentences"] = get_flat_sentences(sub_df["lemmatized_sentences"])
    authors_dicts.append(authors_dict)
    
authors_df = pd.DataFrame(authors_dicts)
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences
0,tlg0031matt,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8658,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1,tlg0031mark,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5298,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18291,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8221,1339,{'0.5': 1},"[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ...","[[εἰμί, ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, ..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."


In [192]:
clean_titles = authors_df["titles"].apply(lambda cell: [title.rpartition(" - ")[2] for title in cell]) 
authors_df["titles"] = clean_titles

In [252]:
def fill_author(author, titles):
    if author == None:
        return titles[0]
    else:
        return author.partition(",")[0]
 
clean_authors = authors_df.apply(lambda row: fill_author(row["author"], row["titles"]), axis=1)
authors_df["author"] = clean_authors

In [253]:
authors_df.loc[authors_df["author_id"]=="tlg0031rest", "author"] = "GNT rest" #, ["author"]] = 

In [254]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,...,dikaiosyne_N,dikaiosyne_TF,G_sents,G_lems,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF,dik_sents_TFIDF_10
20,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,979,212,"{'0.5': 0.5, '1.5': 0.5}","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",...,8,0.008172,"(ξρεσξεντεμ, περιέχω, τολεραντια, :, ἀγαπάω, μ...","(Χριστός, Ἰησοῦς, εἰμί, πᾶς, :, κύριος, ἀφίημι...","(μωμοσκοπεῖται, φιλαργυρία, ἐντυγχανούσας, ἐντ...","(Χριστός, Ἰησοῦς, εἰμί, πᾶς, :, κύριος, ἀφίημι...","[μακάριος, πτωχός, διώκω, εἰμί, βασίλειον, θεο...","[(εἰμί, 0.04444), (οὗτος, 0.04444), (ἐντολή, 0...","[(οὗτος, 0.17646), (ἐντολή, 0.17646), (πτωχός,...","[οὗτος, ἐντολή, πτωχός, διώκω, βασίλειον, θεοῦ..."
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",...,48,0.004319,"(ἄτακτος, βεβαιόω, τυφλός, καθοράω, κληρονομία...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","(χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[θεός, ἀποκαλύπτω, πίστις, πίστις, γράφω, δίκα...","[(θεός, 0.04712), (πίστις, 0.04188), (νομός, 0...","[(πίστις, 0.12845), (ἁμαρτία, 0.1034), (νομός,...","[πίστις, ἁμαρτία, νομός, λογίζομαι, θεός, βασι..."
13,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6632,1410,{'0.5': 1},"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",...,21,0.003166,"(ἐμπορία, περιέχω, ἔξοχος, βεβαιόω, χρυσός, :,...","(:, λέγω, Χριστός, Ἰησοῦς, κύριος, εἶπον, πνεῦ...","(χριστός, ἔχω, ποιέω, λαμβάνω, δίδωμι, κύριος,...","(:, λέγω, Χριστός, Ἰησοῦς, κύριος, εἶπον, γράφ...","[οὗτος, ἄπειμι, εἰρήνη, ἀπολιμπάνω, ἕκαστος, φ...","[(θεός, 0.02791), (πίστις, 0.02326), (γίγνομαι...","[(καυχάομαι, 0.10118), (πίστις, 0.09897), (θεό...","[καυχάομαι, πίστις, θεός, δύσις, πύλη, ἀγάπη, ..."
10,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1398,240,"{'2.5': 0.5, '4.5': 0.5}","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",...,4,0.002861,"(καταγέλαστος, προσδοκάω, τρυφή, πεφανέρωτο, ἄ...","(Χριστιανός, κόσμος, γνῶσις, ξύλον, θεός, οὗτο...","(ἀγαπάω, μόνος, πολύς, πρόσκαιρος, κρυβῇ, ὑπομ...",(δικαιοσύνη),"[ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, ἀδικία, και...","[(ἐκεῖνος, 0.05405), (πολύς, 0.05405), (πῦρ, 0...","[(ἐκεῖνος, 0.25878), (πῦρ, 0.20925), (πολύς, 0...","[ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀν..."
12,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3485,875,"{'0.5': 0.5, '1.5': 0.5}","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",...,9,0.002582,"(οἰκοδομήσουσιν, ὀρός, περιέχω, ἁγιάσατε, ὑποπ...","(κύριος, λέγω, προφήτης, εἶδον, θεός, οὗτος, υ...","(ἀπόλλυμι, ἀκούω, πᾶς, οὗτος, πνεῦμα, ἀσεβής, ...","(κύριος, λέγω, προφήτης, εἶδον, θεός, οὗτος, υ...","[πείθω, οὗτος, σύνοιδα, λαλέω, πολύς, ἐπίσταμα...","[(ὁδός, 0.0411), (κύριος, 0.0274), (θεός, 0.02...","[(ὁδός, 0.17298), (σκότος, 0.13651), (ἀγαθός, ...","[ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συν..."
16,tlg1419,Hermas,1,[The Shepherd of Hermas],[tlg1419.tlg001.1st1K-grc1.xml],30282,15088,2959,{'1.5': 1},"[[τρέφω, πιπράσκω, ῥόδῃ, τὶς, Ῥώμη, πολύς, ἔτη...",...,32,0.002121,"(προδηλῶσαί, τραδυντ, ἄχρι, ἐπερίψατε, ἀναδράμ...","(:, λέγω, κύριος, φημί, οὗτος, ἀκούω, οἰκοδομή...","(ἄγγελος, ἔργον, ἔχω, λαμβάνω, κύριος, εἰμί, τ...","(:, λέγω, κύριος, φημί, οὗτος, ἀκούω, ἔχω, εἰμ...","[ῥέω, προηγουμένοις, ἐκκλησία, κατορθώσωνται, ...","[(ἄγγελος, 0.03279), (:, 0.02623), (ἐργάζομαι,...","[(ἄγγελος, 0.12229), (ἀρετή, 0.10464), (ἐργάζο...","[ἄγγελος, ἀρετή, ἐργάζομαι, ἔργον, ἀλήθεια, πο..."
5,tlg0031pspa,Pauline literature,6,"[Ephesians, Colossians, 2 Thessalonians, 1

In [207]:
# COUNT δικαιοσύνη IN SENTENCES
def count_term_in_sents(lemmatized_sentences):
    return sum([sent.count("δικαιοσύνη") for sent in lemmatized_sentences])

authors_df["dikaiosyne_N"]  = authors_df["lemmatized_sentences"].apply(count_term_in_sents)

In [208]:
authors_df["dikaiosyne_TF"]  = authors_df.apply(lambda row: row["dikaiosyne_N"] / row["lemmata_wordcount"], axis=1)
authors_df.sort_values("dikaiosyne_TF", ascending=False, inplace=True)

In [209]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences,dikaiosyne_N,dikaiosyne_TF
20,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,979,212,"{'0.5': 0.5, '1.5': 0.5}","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",8,0.008172
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",48,0.004319
13,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6632,1410,{'0.5': 1},"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...","[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",21,0.003166
10,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1398,240,"{'2.5': 0.5, '4.5': 0.5}","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",4,0.002861
12,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3485,875,"{'0.5': 0.5, '1.5': 0.5}","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",9,0.002582
16,tlg1419,"Hermas, 2nd cent.",1,[The Shepherd of Hermas],[tlg1419.tlg001.1st1K-grc1.xml],30282,15088,2959,{'1.5': 1},"[[τρέφω, πιπράσκω, ῥόδῃ, τὶς, Ῥώμη, πολύς, ἔτη...","[[τρέφω, πιπράσκω, ῥόδῃ, τὶς, Ῥώμη], [πολύς, ἔ...",32,0.002121
5,tlg0031pspa,Pauline literature,6,"[Ephesians, Colossians, 2 Thessalonians, 1 Tim...","[tlg0031.tlg010.perseus-grc2.xml, tlg0031.tlg0...",8324,4115,390,{'0.5': 1},"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...","[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",8,0.001944
14,tlg1311,Anonymus,1,[Didache XII Apostolorum],[tlg1311.tlg001.1st1K-grc1.xml],2377,1132,284,{'1.5': 1},"[[ὁδός, δύο, εἰμί, εἷς, ζωή, εἷς, θάνατος, δια...","[[ὁδός, δύο, εἰμί, εἷς, ζωή, εἷς, θάνατος, δια...",2,0.001767
6,tlg0031rest,GNT rest,6,"[Hebrews, James, 1 Peter, 2 Peter, Jude, Revel...","[tlg0031.tlg019.perseus-grc2.xml, tlg0031.tlg0...",19782,9798,1037,{'0.5': 1},"[[πολυμερής, καί, πολύτροπος, θεός, λαλέω, πατ...","[[πολυμερής, καί, πολύτροπος, θεός, λαλέω, πατ...",17,0.001735
9,tlg0645,Justinus Martyr,3,"[Apologia, Apology II, Dialogus cum Tryphone]","[tlg0645.tlg001.opp-grc1.xml, tlg0645.tlg002.p...",73449,33852,5829,{'1.5': 1},"[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,...","[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,...",45,0.001329


# Networks for authors

In [213]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [216]:
nx_from_sentences = authors_df["lemmatized_sentences"].apply(textnet.network_from_sentences)
nx_from_lemmata = authors_df["lemmata_lists"].apply(textnet.network_from_lemmata_lists)

In [224]:
authors_df["G_sents"] = nx_from_sentences
authors_df["G_lems"] = nx_from_lemmata

In [225]:
def ego_network(G):
    try:
        G_ego = textnet.construct_ego_network(G, "δικαιοσύνη", 30, reduced=False)
    except:
        G_ego = nx.Graph()
        G_ego.clear()
    return G_ego

In [226]:
authors_df["dikaiosyne_G_sents"] = authors_df["G_sents"].apply(ego_network)
authors_df["dikaiosyne_G_lems"] = authors_df["G_lems"].apply(ego_network)

In [227]:
list(authors_df.iloc[0]["dikaiosyne_G_sents"].nodes())

['μωμοσκοπεῖται',
 'φιλαργυρία',
 'ἐντυγχανούσας',
 'ἐντολή',
 'πρέσβυς',
 'θυσιαστήριον',
 'πᾶς',
 'εὔσπλαγχνος',
 'χαλιναγωγοῦντες',
 'οὗτος',
 'γινωσκούσας',
 'ὀφειλέτης',
 'θεός',
 'ἁγνεία',
 'ὀργή',
 'λανθάνω',
 'ἄμεμπτος',
 'διαβολή',
 'δικαιοσύνη',
 'καταλαλιά',
 'προσωποληψίας',
 'κύριος',
 'εἰμί',
 'πίστις',
 'οὐδείς',
 'κακός',
 'προνοοῦντες',
 'κρίσις',
 'ψευδομαρτυρία',
 'ἀπεχόμενοι',
 'ἁμαρτία']

# Terms within sentences

In [228]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        flatlist = [word for word in flatlist if word != "δικαιοσύνη"]
        return flatlist
    else: 
        return sentences_with_term
authors_df["dik_sents_flat"] = authors_df["lemmatized_sentences"].apply(lambda cell: sentences_with_term(cell, "δικαιοσύνη", flatlist=True))

In [229]:
def get_tf(lemmata):
    fd_list = list(nltk.FreqDist(lemmata).most_common())
    tf_list = [(tup[0], np.round(tup[1] / len(lemmata), 5)) for tup in fd_list]
    return tf_list

authors_df["dik_sents_TF"] = authors_df["dik_sents_flat"].apply(get_tf)

In [230]:
def term_idf(term, all_sentences):
    idf_raw =  len(all_sentences) / len([sent for sent in all_sentences if term in sent])
    idf_log = math.log(idf_raw)
    return idf_log

def get_tfidf(key_term_lemmata, all_sentences):
    fd_list = list(nltk.FreqDist(key_term_lemmata).most_common())
    tfidf_list = [(tup[0], np.round((tup[1] / len(key_term_lemmata) * term_idf(tup[0], all_sentences)), 5)) for tup in fd_list]
    tfidf_list = sorted(tfidf_list, key=lambda tup: tup[1], reverse=True)
    return tfidf_list

authors_df["dik_sents_TFIDF"] = authors_df.apply(lambda x: get_tfidf(x["dik_sents_flat"], x["lemmatized_sentences"]), axis=1)

In [231]:
authors_df["dik_sents_TFIDF_10"] = authors_df["dik_sents_TFIDF"].apply(lambda cell: [tup[0] for tup in cell[:10]])

In [232]:
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,...,dikaiosyne_N,dikaiosyne_TF,G_sents,G_lems,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF,dik_sents_TFIDF_10
20,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,979,212,"{'0.5': 0.5, '1.5': 0.5}","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",...,8,0.008172,"(ξρεσξεντεμ, περιέχω, τολεραντια, :, ἀγαπάω, μ...","(Χριστός, Ἰησοῦς, εἰμί, πᾶς, :, κύριος, ἀφίημι...","(μωμοσκοπεῖται, φιλαργυρία, ἐντυγχανούσας, ἐντ...","(Χριστός, Ἰησοῦς, εἰμί, πᾶς, :, κύριος, ἀφίημι...","[μακάριος, πτωχός, διώκω, εἰμί, βασίλειον, θεο...","[(εἰμί, 0.04444), (οὗτος, 0.04444), (ἐντολή, 0...","[(οὗτος, 0.17646), (ἐντολή, 0.17646), (πτωχός,...","[οὗτος, ἐντολή, πτωχός, διώκω, βασίλειον, θεοῦ..."
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11113,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",...,48,0.004319,"(ἄτακτος, βεβαιόω, τυφλός, καθοράω, κληρονομία...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","(χριστός, ἔχω, νομός, δίδωμι, κύριος, ἔθνος, ε...","(Χριστός, Ἰησοῦς, κύριος, εἰμί, θεός, χάρις, π...","[θεός, ἀποκαλύπτω, πίστις, πίστις, γράφω, δίκα...","[(θεός, 0.04712), (πίστις, 0.04188), (νομός, 0...","[(πίστις, 0.12845), (ἁμαρτία, 0.1034), (νομός,...","[πίστις, ἁμαρτία, νομός, λογίζομαι, θεός, βασι..."
13,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6632,1410,{'0.5': 1},"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",...,21,0.003166,"(ἐμπορία, περιέχω, ἔξοχος, βεβαιόω, χρυσός, :,...","(:, λέγω, Χριστός, Ἰησοῦς, κύριος, εἶπον, πνεῦ...","(χριστός, ἔχω, ποιέω, λαμβάνω, δίδωμι, κύριος,...","(:, λέγω, Χριστός, Ἰησοῦς, κύριος, εἶπον, γράφ...","[οὗτος, ἄπειμι, εἰρήνη, ἀπολιμπάνω, ἕκαστος, φ...","[(θεός, 0.02791), (πίστις, 0.02326), (γίγνομαι...","[(καυχάομαι, 0.10118), (πίστις, 0.09897), (θεό...","[καυχάομαι, πίστις, θεός, δύσις, πύλη, ἀγάπη, ..."
10,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1398,240,"{'2.5': 0.5, '4.5': 0.5}","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",...,4,0.002861,"(καταγέλαστος, προσδοκάω, τρυφή, πεφανέρωτο, ἄ...","(Χριστιανός, κόσμος, γνῶσις, ξύλον, θεός, οὗτο...","(ἀγαπάω, μόνος, πολύς, πρόσκαιρος, κρυβῇ, ὑπομ...",(δικαιοσύνη),"[ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, ἀδικία, και...","[(ἐκεῖνος, 0.05405), (πολύς, 0.05405), (πῦρ, 0...","[(ἐκεῖνος, 0.25878), (πῦρ, 0.20925), (πολύς, 0...","[ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀν..."
12,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3485,875,"{'0.5': 0.5, '1.5': 0.5}","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",...,9,0.002582,"(οἰκοδομήσουσιν, ὀρός, περιέχω, ἁγιάσατε, ὑποπ...","(κύριος, λέγω, προφήτης, εἶδον, θεός, οὗτος, υ...","(ἀπόλλυμι, ἀκούω, πᾶς, οὗτος, πνεῦμα, ἀσεβής, ...","(κύριος, λέγω, προφήτης, εἶδον, θεός, οὗτος, υ...","[πείθω, οὗτος, σύνοιδα, λαλέω, πολύς, ἐπίσταμα...","[(ὁδός, 0.0411), (κύριος, 0.0274), (θεός, 0.02...","[(ὁδός, 0.17298), (σκότος, 0.13651), (ἀγαθός, ...","[ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συν..."


In [233]:
authors_df.columns

Index(['author_id', 'author', 'n_titles', 'titles', 'filenames', 'wordcount',
       'lemmata_wordcount', 'n_sentences', 'date_probs', 'lemmata_lists',
       'lemmatized_sentences', 'dikaiosyne_N', 'dikaiosyne_TF', 'G_sents',
       'G_lems', 'dikaiosyne_G_sents', 'dikaiosyne_G_lems', 'dik_sents_flat',
       'dik_sents_TF', 'dik_sents_TFIDF', 'dik_sents_TFIDF_10'],
      dtype='object')

In [300]:
columns_to_omit = ["lemmatized_sentences", "lemmata_lists", "G_sents", "G_lems", "dikaiosyne_G_sents" , "dikaiosyne_G_lems"]

In [301]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("authors_df_20201202", 1,1), authors_df.drop(columns_to_omit, axis=1))

# N of shared terms within dik_sents_TFIDF_10

In [236]:
authors_df["dik_sents_TFIDF_10"].tolist()[:5]

[['οὗτος',
  'ἐντολή',
  'πτωχός',
  'διώκω',
  'βασίλειον',
  'θεοῦ.',
  'ἀδελφός',
  'ἐπιτρέπω',
  'προεπεκαλέσασθέ',
  'πληρόω'],
 ['πίστις',
  'ἁμαρτία',
  'νομός',
  'λογίζομαι',
  'θεός',
  'βασιλεύω',
  'πιστεύω',
  'ἀνομία',
  'Χριστός',
  'δοῦλος'],
 ['καυχάομαι',
  'πίστις',
  'θεός',
  'δύσις',
  'πύλη',
  'ἀγάπη',
  'σώζω',
  'πατήρ',
  'ὑπομονή',
  'βασίλεια'],
 ['ἐκεῖνος',
  'πῦρ',
  'πολύς',
  'ἐφηδόμενος',
  'ἁμάρτημα',
  'ἀνεχόμενος',
  'ἀδικία',
  'συνευδοκέω',
  'ἐλεγχθέντες',
  'ἀνάξιος'],
 ['ὁδός',
  'σκότος',
  'ἀγαθός',
  'σύνοιδα',
  'ἐπίσταμαι',
  'συνώδευσεν',
  'ἀναγκάζω',
  'ἀγαπάζω',
  'ἀνατελεῖ',
  'προπορεύσεται']]

In [240]:
l1 = ['καυχάομαι',
  'πίστις',
  'θεός',
  'δύσις',
  'πύλη',
  'ἀγάπη',
  'σώζω',
  'πατήρ',
  'ὑπομονή',
  'βασίλεια']

l2 =  ['πίστις',
  'ἁμαρτία',
  'νομός',
  'λογίζομαι',
  'θεός',
  'βασιλεύω',
  'πιστεύω',
  'ἀνομία',
  'Χριστός',
  'δοῦλος']

len(set(l1) & set(l2))

2

In [408]:
edges_tuples = []
for author1 in authors_df["author"].tolist():
    author1_words = authors_df.loc[authors_df["author"]==author1, "dik_sents_TFIDF_10"].values[0]
    for author2 in authors_df["author"].tolist():
        if author2 != author1:
            author2_words = authors_df.loc[authors_df["author"]==author2, "dik_sents_TFIDF_10"].values[0]
            shared = len(set(author1_words) & set(author2_words))
            if shared > 1:
                edges_tuples.append((author1, author2, shared))

In [409]:
np.array(edges_tuples)

array([['Paul of Tarsus', 'Clemens Romanus', '2'],
       ['Paul of Tarsus', 'Justinus Martyr', '2'],
       ['Paul of Tarsus', 'Clement of Alexandria', '2'],
       ['Paul of Tarsus', 'Origenes', '2'],
       ['Paul of Tarsus', 'Theophilus Antiochenus', '3'],
       ['Paul of Tarsus', 'Eusebius', '2'],
       ['Paul of Tarsus', 'Epiphanius', '5'],
       ['Clemens Romanus', 'Paul of Tarsus', '2'],
       ['Clemens Romanus', 'Epiphanius', '2'],
       ['Hermas', 'Pauline literature', '2'],
       ['Hermas', 'Clement of Alexandria', '3'],
       ['Hermas', 'Origenes', '2'],
       ['Hermas', 'Eusebius', '2'],
       ['Hermas', 'Epiphanius', '2'],
       ['Pauline literature', 'Hermas', '2'],
       ['Pauline literature', 'GNT rest', '2'],
       ['Anonymus', 'Athanasius of Alexandria', '2'],
       ['GNT rest', 'Pauline literature', '2'],
       ['GNT rest', 'Gregorius Nazianzenus', '2'],
       ['Justinus Martyr', 'Paul of Tarsus', '2'],
       ['Justinus Martyr', 'Athenagoras', '2'],


In [293]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [410]:
G = nx.Graph()
G.clear()
G.add_weighted_edges_from(edges_tuples)
for (u, v, wt) in G.edges.data('weight'):
    G[u][v]["weight"] = int(wt)
total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)
G.remove_edges_from(nx.selfloop_edges(G))

In [412]:
G.edges.data()

EdgeDataView([('Paul of Tarsus', 'Clemens Romanus', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Paul of Tarsus', 'Justinus Martyr', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Paul of Tarsus', 'Clement of Alexandria', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Paul of Tarsus', 'Origenes', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Paul of Tarsus', 'Theophilus Antiochenus', {'weight': 3, 'norm_weight': 0.03371, 'distance': 0.33333, 'norm_distance': 29.66479}), ('Paul of Tarsus', 'Eusebius', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Paul of Tarsus', 'Epiphanius', {'weight': 5, 'norm_weight': 0.05618, 'distance': 0.2, 'norm_distance': 17.79993}), ('Clemens Romanus', 'Epiphanius', {'weight': 2, 'norm_weight': 0.02247, 'distance': 0.5, 'norm_distance': 44.50378}), ('Justinus Marty

In [423]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [424]:
textnet.draw_2d_network(G)

In [425]:
textnet.draw_3d_network(G)

In [426]:
G_paul = textnet.construct_ego_network(G, "Paul of Tarsus", 10, reduced=False)

In [427]:
textnet.draw_2d_network(G_paul)

In [429]:
nx.to_pandas_adjacency(G_paul).astype()

,Paul of Tarsus,Clemens Romanus,Justinus Martyr,Clement of Alexandria,Origenes,Theophilus Antiochenus,Eusebius,Epiphanius,Hermas,Athenagoras,Basil
Paul of Tarsus,0.0,2.0,2.0,2.0,2.0,3.0,2.0,5.0,0.0,0.0,0.0
Clemens Romanus,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Justinus Martyr,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0
Clement of Alexandria,2.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,3.0,4.0,0.0
Origenes,2.0,0.0,0.0,3.0,0.0,2.0,2.0,2.0,2.0,0.0,3.0
Theophilus Antiochenus,3.0,0.0,0.0,2.0,2.0,0.0,2.0,3.0,0.0,0.0,2.0
Eusebius,2.0,0.0,2.0,3.0,2.0,2.0,0.0,3.0,2.0,0.0,0.0
Epiphanius,5.0,2.0,0.0,4.0,2.0,3.0,3.0,0.0,2.0,0.0,0.0
Hermas,0.0,0.0,0.0,3.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0
Athenagoras,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  vectorization based on tfidf values

In [306]:
#tup_to_dict 
tfidfs_df_raw = pd.DataFrame([dict(tfidfs) for tfidfs in authors_df["dik_sents_TFIDF"].tolist()])
tfidfs_df_raw

,οὗτος,ἐντολή,πτωχός,διώκω,βασίλειον,θεοῦ.,ἀδελφός,ἐπιτρέπω,προεπεκαλέσασθέ,πληρόω,...,δειλία,ὑπερβάλλω,προειρημένου,παρακολουθέω,τελευτάω,Ἰωσήφ,ἀπεργάζομαι,τέσσαρες,ἡττηθέντα,κᾀκεῖνον
0,0.17646,0.17646,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00721,NaN,NaN,0.03540,0.01755,0.01038,NaN,NaN,NaN,0.01286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.03035,NaN,NaN,0.02539,NaN,0.02145,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.04409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.04496,0.03655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,0.07788,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.00989,0.01878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02413,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.01378,0.01545,0.03205,NaN,NaN,NaN,NaN,NaN,NaN,0.01582,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
selected_columns = []
for column in tfidfs_df_raw.columns:
    if len(tfidfs_df_raw[tfidfs_df_raw[column].notnull()]) > 1:
        selected_columns.append(column)
len(selected_columns)

871

In [322]:
tfidfs_df_filtered = tfidfs_df_raw[selected_columns]
tfidfs_df_filtered.fillna(0, inplace=True)

In [323]:
tfidfs_df_filtered

,οὗτος,ἐντολή,πτωχός,διώκω,βασίλειον,θεοῦ.,ἀδελφός,πληρόω,ἐκφέρω,ὅπλον,...,κλῆσις,τιμωρία,ὁ,θεοφιλής,ὁμολογήσαντα,θελήσαντα,δουλεία,πατριάρχης,δύο,Μαρία
0,0.17646,0.17646,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,0.11904,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00721,0.00000,0.00000,0.03540,0.01755,0.01038,0.00000,0.01286,0.00000,0.04721,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.03035,0.00000,0.00000,0.02539,0.00000,0.02145,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.04409,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.04496,0.03655,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
6,0.00000,0.00000,0.00000,0.07788,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
8,0.00989,0.01878,0.00000,0.00000,0.00000,0.00000,0.00000,0.02413,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.01378,0.01545,0.03205,0.00000,0.00000,0.00000,0.00000,0.01582,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [317]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity

In [349]:
tfidfs_sim_mat = cosine_similarity(tfidfs_df_filtered.to_numpy()).round(5)
tfidfs_dist_mat = 1 - cosine_similarity(tfidfs_df_filtered.to_numpy())
tfidfs_dist_mat = tfidfs_dist_mat.round(5)

In [350]:
tfidfs_sim_mat_df = pd.DataFrame(tfidfs_sim_mat, columns=authors_df["author"].tolist(), index=authors_df["author"].tolist())
tfidfs_dist_mat_df = pd.DataFrame(tfidfs_dist_mat, columns=authors_df["author"].tolist(), index=authors_df["author"].tolist())

tfidfs_sim_mat_df

,Polycarp,Paul of Tarsus,Clemens Romanus,Pseudo-Justinus Martyr,Barnabas,Hermas,Pauline literature,Anonymus,GNT rest,Justinus Martyr,...,Luke (the evangelist),Ignatius Antiochenus,Basil,Maximus of Tyre,Hippolytus,Irenaeus,Tatianus,Anonymous,Mark,Dionysius Corinthius
Polycarp,1.00000,0.22233,0.24048,0.00707,0.06487,0.23874,0.13808,0.03069,0.16175,0.12134,...,0.10045,0.15804,0.04796,0.06930,0.02362,0.04066,0.0,0.0,0.0,0.0
Paul of Tarsus,0.22233,1.00000,0.36419,0.19759,0.18910,0.24224,0.21859,0.04052,0.35578,0.26018,...,0.18337,0.02249,0.20511,0.07866,0.07601,0.03729,0.0,0.0,0.0,0.0
Clemens Romanus,0.24048,0.36419,1.00000,0.08621,0.19873,0.30964,0.29774,0.06398,0.28855,0.30940,...,0.25031,0.01146,0.19861,0.05928,0.06484,0.09617,0.0,0.0,0.0,0.0
Pseudo-Justinus Martyr,0.00707,0.19759,0.08621,1.00000,0.06196,0.13698,0.08032,0.00000,0.16373,0.12957,...,0.03775,0.00000,0.12857,0.11074,0.02004,0.00000,0.0,0.0,0.0,0.0
Barnabas,0.06487,0.18910,0.19873,0.06196,1.00000,0.18740,0.13178,0.37203,0.18130,0.51070,...,0.16217,0.00000,0.08399,0.06339,0.01402,0.04351,0.0,0.0,0.0,0.0
Hermas,0.23874,0.24224,0.30964,0.13698,0.18740,1.00000,0.25890,0.13020,0.27019,0.22457,...,0.21115,0.02028,0.19934,0.09997,0.09835,0.11337,0.0,0.0,0.0,0.0
Pauline literature,0.13808,0.21859,0.29774,0.08032,0.13178,0.25890,1.00000,0.11201,0.24683,0.21356,...,0.15811,0.00000,0.17862,0.02523,0.04035,0.02878,0.0,0.0,0.0,0.0
Anonymus,0.03069,0.04052,0.06398,0.00000,0.37203,0.13020,0.11201,1.00000,0.02701,0.11055,...,0.08372,0.00000,0.05254,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0
GNT rest,0.16175,0.35578,0.28855,0.16373,0.18130,0.27019,0.24683,0.02701,1.00000,0.31788,...,0.20377,0.05106,0.16022,0.04254,0.04532,0.03687,0.0,0.0,0.0,0.0
Justinus Martyr,0.12134,0.26018,0.30940,0.12957,0.51070,0.22457,0.21356,0.11055,0.31788,1.00000,...,0.25424,0.02754,0.24055,0.09952,0.08714,0.07255,0.0,0.0,0.0,0.0


In [360]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("tfidfs_sim", 1, 1), tfidfs_sim_mat_df, include_index=True)

In [351]:
tfidfs_dist_mat_df

,Polycarp,Paul of Tarsus,Clemens Romanus,Pseudo-Justinus Martyr,Barnabas,Hermas,Pauline literature,Anonymus,GNT rest,Justinus Martyr,...,Luke (the evangelist),Ignatius Antiochenus,Basil,Maximus of Tyre,Hippolytus,Irenaeus,Tatianus,Anonymous,Mark,Dionysius Corinthius
Polycarp,0.00000,0.77767,0.75952,0.99293,0.93513,0.76126,0.86192,0.96931,0.83825,0.87866,...,0.89955,0.84196,0.95204,0.93070,0.97638,0.95934,1.0,1.0,1.0,1.0
Paul of Tarsus,0.77767,-0.00000,0.63581,0.80241,0.81090,0.75776,0.78141,0.95948,0.64422,0.73982,...,0.81663,0.97751,0.79489,0.92134,0.92399,0.96271,1.0,1.0,1.0,1.0
Clemens Romanus,0.75952,0.63581,0.00000,0.91379,0.80127,0.69036,0.70226,0.93602,0.71145,0.69060,...,0.74969,0.98854,0.80139,0.94072,0.93516,0.90383,1.0,1.0,1.0,1.0
Pseudo-Justinus Martyr,0.99293,0.80241,0.91379,-0.00000,0.93804,0.86302,0.91968,1.00000,0.83627,0.87043,...,0.96225,1.00000,0.87143,0.88926,0.97996,1.00000,1.0,1.0,1.0,1.0
Barnabas,0.93513,0.81090,0.80127,0.93804,-0.00000,0.81260,0.86822,0.62797,0.81870,0.48930,...,0.83783,1.00000,0.91601,0.93661,0.98598,0.95649,1.0,1.0,1.0,1.0
Hermas,0.76126,0.75776,0.69036,0.86302,0.81260,0.00000,0.74110,0.86980,0.72981,0.77543,...,0.78885,0.97972,0.80066,0.90003,0.90165,0.88663,1.0,1.0,1.0,1.0
Pauline literature,0.86192,0.78141,0.70226,0.91968,0.86822,0.74110,0.00000,0.88799,0.75317,0.78644,...,0.84189,1.00000,0.82138,0.97477,0.95965,0.97122,1.0,1.0,1.0,1.0
Anonymus,0.96931,0.95948,0.93602,1.00000,0.62797,0.86980,0.88799,0.00000,0.97299,0.88945,...,0.91628,1.00000,0.94746,1.00000,1.00000,1.00000,1.0,1.0,1.0,1.0
GNT rest,0.83825,0.64422,0.71145,0.83627,0.81870,0.72981,0.75317,0.97299,-0.00000,0.68212,...,0.79623,0.94894,0.83978,0.95746,0.95468,0.96313,1.0,1.0,1.0,1.0
Justinus Martyr,0.87866,0.73982,0.69060,0.87043,0.48930,0.77543,0.78644,0.88945,0.68212,0.00000,...,0.74576,0.97246,0.75945,0.90048,0.91286,0.92745,1.0,1.0,1.0,1.0


In [379]:
G = nx.from_numpy_matrix(tfidfs_sim_mat)

authors_dict = dict(zip(range(len(authors_df["author"].tolist())), authors_df["author"].tolist()))
G = nx.relabel_nodes(G, authors_dict)

total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)

edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

In [383]:
sim_threshold = 0.3
edges_to_remove = []
for edge in G.edges:
    if G[edge[0]][edge[1]]["weight"] < sim_threshold:
        edges_to_remove.append(edge)
for element in edges_to_remove:
    G.remove_edge(element[0], element[1])

G.remove_nodes_from(list(nx.isolates(G)))

In [384]:
G.edges.data()

EdgeDataView([('Paul of Tarsus', 'Clemens Romanus', {'weight': 0.36419, 'norm_weight': 0.01401, 'distance': 2.74582, 'norm_distance': 71.37759}), ('Paul of Tarsus', 'GNT rest', {'weight': 0.35578, 'norm_weight': 0.01368, 'distance': 2.81073, 'norm_distance': 73.09942}), ('Paul of Tarsus', 'Clement of Alexandria', {'weight': 0.37199, 'norm_weight': 0.01431, 'distance': 2.68824, 'norm_distance': 69.8812}), ('Paul of Tarsus', 'Origenes', {'weight': 0.45456, 'norm_weight': 0.01748, 'distance': 2.19993, 'norm_distance': 57.20824}), ('Paul of Tarsus', 'Theophilus Antiochenus', {'weight': 0.3039, 'norm_weight': 0.01169, 'distance': 3.29056, 'norm_distance': 85.5432}), ('Paul of Tarsus', 'Eusebius', {'weight': 0.38956, 'norm_weight': 0.01498, 'distance': 2.567, 'norm_distance': 66.75567}), ('Paul of Tarsus', 'Epiphanius', {'weight': 0.49057, 'norm_weight': 0.01887, 'distance': 2.03845, 'norm_distance': 52.99417}), ('Clemens Romanus', 'Hermas', {'weight': 0.30964, 'norm_weight': 0.01191, 'dista

In [389]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [390]:
textnet.draw_2d_network(G)